In [1]:
import sys
from pathlib import Path
import os
from langchain_openai import ChatOpenAI
import concurrent.futures

current_dir = Path.cwd()
agents_dir = current_dir / 'agents'
tools_dir = current_dir / 'tools'
sys.path.append(str(agents_dir.resolve()))
sys.path.append(str(tools_dir.resolve()))
os.environ["OPENAI_API_KEY"] = "sk-_Sm2Yty66t_I5qeS3kYSC5KcAe1JvwqyOOL75ijRzJT3BlbkFJK9o9tQ6pmlsDI6u6s7cQJIUlSJHe5hpGsmugSrh0kA"

from risk_advisor import analyze_company_risk
from efficiency_advisor import analyze_company_efficiency
from growth_advisor import analyze_company_growth
from management_advisor import analyze_company_management
from valuation_advisor import analyze_company_valuation
from finance_advisor import analyze_company_finance

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

def save_report(report_content, file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        if isinstance(report_content, dict):
            f.write(report_content.get('text', str(report_content)))
        else:
            f.write(str(report_content))
    print(f"Report saved to: {file_path}")

# レポート生成用プロンプト作成
summary_template = """
**Target Company:** {target_company}

---

**[Task Description]**

As an M&A Genie, your goal is to provide a final acquisition recommendation for the target company. You must synthesize the final recommendations from the six evaluation reports into a comprehensive acquisition strategy. Focus on integrating the key suggestions from each report without reanalyzing them in detail.

**Note:** The output must be formatted in Markdown (MD). Use headings, bold text, and lists appropriately.

---

**[Synthesis Steps]**

1. **Risk Assessment**
   - **Review:** Review the final recommendation in the {risk_assessment}.
   - **Analyze:** Identify how the risks influence the acquisition decision.
   - **Summarize (200 words):** Summarize whether the risks support or hinder the acquisition.

2. **Efficiency Assessment**
   - **Review:** Review the final recommendation in the {efficiency_assessment}.
   - **Analyze:** Analyze how operational efficiency impacts the acquisition.
   - **Summarize (200 words):** Summarize the effect of efficiency on the acquisition.

3. **Growth Assessment**
   - **Review:** Review the final recommendation in the {growth_assessment}.
   - **Analyze:** Assess the company’s growth potential and its justification for acquisition.
   - **Summarize (200 words):** Summarize how growth potential affects the acquisition decision.

4. **Management Assessment**
   - **Review:** Review the final recommendation in the {management_assessment}.
   - **Analyze:** Evaluate the management team’s performance and strategy in relation to the acquisition.
   - **Summarize (200 words):** Summarize management’s contribution to the acquisition decision.

5. **Valuation Assessment**
   - **Review:** Review the final recommendation in the {valuation_assessment}.
   - **Analyze:** Assess whether the company’s valuation supports the acquisition.
   - **Summarize (200 words):** Summarize how valuation affects the acquisition decision.

6. **Financial Assessment**
   - **Review:** Review the final recommendation in the {finance_assessment}.
   - **Analyze:** Analyze the company’s financial health and its relevance to the acquisition.
   - **Summarize (200 words):** Summarize how financial health impacts the acquisition decision.

---

**[Final Acquisition Recommendation]**

- **Integration:** Integrate the final recommendations from each report and provide a comprehensive judgment on the company’s suitability for acquisition.
- **Decision:** Provide a final recommendation: **Acquire**, **Wait and Monitor**, or **Do Not Acquire**, and explain how each assessment contributes to the overall decision.

"""

generate_report_prompt = PromptTemplate(
    input_variables=["target_company", "risk_assessment", "efficiency_assessment", "growth_assessment", 
                     "management_assessment", "valuation_assessment", "finance_assessment"],
    template=summary_template
)

# レポート通訳用の ChatPromptTemplate作成し
translate_prompt = ChatPromptTemplate.from_messages([ 
    ("system", 
     """You are a professional translator specializing in financial and corporate documents, with a focus on translating M&A reports. Ensure that the translation is formal, uses precise financial terminology, and is suited for Japanese executives or financial advisors.
     When translating, always keep the following structure:
     - Translate '1. Risk Assessment Summary:' as '1. リスク評価サマリー：'
     - Translate '2. Efficiency Assessment Summary:' as '2. 効率性評価サマリー：'
     - Translate '3. Growth Assessment Summary:' as '3. 成長評価サマリー：'
     - Translate '4. Management Assessment Summary:' as '4. 経営評価サマリー：'
     - Translate '5. Valuation Assessment Summary:' as '5. バリュエーション評価サマリー：'
     - Translate '6. Financial Assessment Summary:' as '6. 財務評価サマリー：'
     - Translate '[Final Acquisition Recommendation]' as '最終アドバイス'
     Ensure that the translation follows this structure, and use local Japanese financial terminology where appropriate。"""
    ),
    ("human", "{input}")
])

# レポート生成用 LLMChain
llm = ChatOpenAI(temperature=0.5, max_tokens=16000, model="gpt-4o-mini")
generate_report_chain = LLMChain(llm=llm, prompt=generate_report_prompt)

# レポート通訳用 LLMChain
translate_chain = LLMChain(llm=llm, prompt=translate_prompt)

# 各種agentの結果を並行処理するための関数

def fetch_assessment_data(target_company):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {
            "risk": executor.submit(analyze_company_risk, target_company),
            "efficiency": executor.submit(analyze_company_efficiency, target_company),
            "growth": executor.submit(analyze_company_growth, target_company),
            "management": executor.submit(analyze_company_management, target_company),
            "valuation": executor.submit(analyze_company_valuation, target_company),
            "finance": executor.submit(analyze_company_finance, target_company)
        }

        results = {key: future.result() for key, future in futures.items()}
    
    return results

# Agentで情報収集/まとめ

def generate_and_save_reports(target_company):
    stock_symbol = target_company
    result_dir = Path.cwd().parent / f"MAadvisor-main/result/{stock_symbol}"
    
    folders = ["Risk", "Efficiency", "Growth", "Management", "Valuation", "Finance", "Final"]
    for folder in folders:
        (result_dir / folder).mkdir(parents=True, exist_ok=True)

    assessments = fetch_assessment_data(target_company)

    # 英語レポートを作成
    final_report = generate_report_chain.invoke({
        "target_company": target_company,
        "risk_assessment": assessments["risk"],
        "efficiency_assessment": assessments["efficiency"],
        "growth_assessment": assessments["growth"],
        "management_assessment": assessments["management"],
        "valuation_assessment": assessments["valuation"],
        "finance_assessment": assessments["finance"]
    })

    if isinstance(final_report, dict):
        final_report = final_report.get('text', str(final_report))

    # 英語レポートを保存
    english_report_path = result_dir / "Final" / f"{stock_symbol}_final_report.md"
    save_report(final_report, english_report_path)

    # 日本語に通訳
    japanese_translation = translate_chain.invoke({
        "input": final_report
    })

    if isinstance(japanese_translation, dict):
        japanese_translation = japanese_translation.get('text', str(japanese_translation))

    # 日本語レポートを保存
    japanese_report_path = result_dir / "Final" / f"{stock_symbol}_final_report_japanese.md"
    save_report(japanese_translation, japanese_report_path)

    return result_dir

import gradio as gr
from pathlib import Path
import sys

current_dir = Path.cwd()
agents_dir = current_dir / 'agents'
tools_dir = current_dir / 'tools'
sys.path.append(str(agents_dir.resolve()))
sys.path.append(str(tools_dir.resolve()))

def read_report(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()
    except FileNotFoundError:
        return f"レポートファイルが見つかりません: {file_path}"
    except Exception as e:
        return f"レポートの読み込み中にエラーが発生しました: {str(e)}"

def analyze_company(stock_symbol):
    generate_and_save_reports(stock_symbol)
    
    result_dir = Path.cwd().parent / f"MAadvisor-main/result/{stock_symbol}/"
    
    reports = [
        f"Risk/{stock_symbol}_risk_analysis_japanese.md",
        f"Efficiency/{stock_symbol}_efficiency_analysis_japanese.md",
        f"Growth/{stock_symbol}_growth_analysis_japanese.md",
        f"Management/{stock_symbol}_management_analysis_japanese.md",
        f"Valuation/{stock_symbol}_valuation_analysis_japanese.md",
        f"Finance/{stock_symbol}_financial_analysis_japanese.md",
        f"Final/{stock_symbol}_final_report_japanese.md"
    ]
    
    return [read_report(result_dir / file_path) for file_path in reports]

import gradio as gr

with gr.Blocks(theme=gr.themes.Soft(), css="""
    #header {
        text-align: center;
        margin-bottom: 20px;
        font-family: 'Roboto', sans-serif;
        color: #f0f0f0;
    }

    #input-box {
        background-color: #2c3e50;
        color: #ffffff;
        border-radius: 10px;
        padding: 12px;
        border: 1px solid #ccc;
    }

    #analyze-button {
        background: linear-gradient(90deg, #4caf50, #2e7d32);
        color: white;
        border-radius: 30px;
        transition: all 0.3s ease;
        padding: 12px 20px;
    }

    #analyze-button:hover {
        box-shadow: 0 4px 12px rgba(0, 0, 0, 0.15);
    }

    .gradio-container {
        max-width: 600px;
        margin: auto;
        padding: 25px;
        background-color: #1c2833;
        border-radius: 12px;
        box-shadow: 0 5px 8px rgba(0, 0, 0, 0.08);
    }

    body {
        background-color: #1c2833;
        color: #ffffff;
    }

    h1, h2, p {
        color: #f0f0f0;
    }

    .markdown-output {
        color: #f0f0f0;
    }

    .gr-tab-item {
        background-color: #2e4053;
        color: #f0f0f0;
    }




""") as demo:

    gr.Markdown("""
    <h1 style="font-size: 2.5em; margin-bottom: 10px;">M&A Genie 🧞‍♂️</h1>
    <p style="font-size: 1.2em;">M&A Genieは、企業の未来を見通し、最適なM&Aの意思決定をお手伝いします✨<br>
    証券コードを入力し、魔法の「分析」ボタンを押して、アドバイスを手に入れましょう🔍💼</p>

    """, elem_id="header")

    with gr.Row():
        input_text = gr.Textbox(
            placeholder="分析したい企業の証券コードを教えてください！🧞‍♂️ (例: 4901.T)",
            show_label=False,
            elem_id="input-box"
        )
        analyze_btn = gr.Button("🔍 分析", elem_id="analyze-button")

    with gr.Tabs():
        with gr.TabItem("📊 リスク評価"):
            risk_output = gr.Markdown()
        with gr.TabItem("⚙️ 効率性評価"):
            efficiency_output = gr.Markdown()
        with gr.TabItem("🌱 成長性評価"):
            growth_output = gr.Markdown()
        with gr.TabItem("🏢 経営評価"):
            management_output = gr.Markdown()
        with gr.TabItem("💰 バリュエーション評価"):
            valuation_output = gr.Markdown()
        with gr.TabItem("📈 財務評価"):
            financial_output = gr.Markdown()
        with gr.TabItem("📝 最終報告"):
            final_output = gr.Markdown()

    outputs = [risk_output, efficiency_output, growth_output,
               management_output, valuation_output, financial_output,
               final_output]

    analyze_btn.click(
        analyze_company,
        inputs=input_text,
        outputs=outputs
    )

demo.launch()

C:\Users\evanx\AppData\Local\Temp\ipykernel_126424\1676835360.py:113: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(temperature=0.5, max_tokens=16000, model="gpt-4o-mini")
C:\Users\evanx\AppData\Local\Temp\ipykernel_126424\1676835360.py:114: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  generate_report_chain = LLMChain(llm=llm, prompt=generate_report_prompt)
e:\anaconda\envs\kaggle\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import t

Running on local URL:  http://127.0.0.1:7860

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB

To create a public link, set `share=True` in `launch()`.


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `MA_Info_Tool` with `{'symbol': '4901.T'}`



Invoking: `MA_Info_Tool` with `{'symbol': '4901.T'}`



Invoking: `MA_Info_Tool` with `{'symbol': '4901.T'}`



Invoking: `MA_Info_Tool` with `{'symbol': '4901.T'}`



Invoking: `MA_Info_Tool` with `{'symbol': '4901.T'}`


経営陣情報：（FUJIFILM Holdings Corporation）
氏名: Mr. Kenji  Sukeno
  役職: Executive Chairman of The Board
  年齢: 69
  生年: 1954
  報酬: 147000000

氏名: Mr. Teiichi  Goto
  役職: President, CEO & Director
  年齢: 64
  生年: 1959
  報酬: 127000000

氏名: Mr. Masayuki  Higuchi
  役職: Corporate VP, GM of Corporate Planning Division, Accounting and Finance Div, CFO & Director
  年齢: 60
  生年: 1963
  報酬: 37000000

氏名: Ms. Chisato  Yoshizawa
  役職: Corporate VP, GM of Corp Communications and ESG Division-Global Audit Dep, Brand Mgmt & Director
  年齢: 60
  生年: 1963
  報酬: 37000000

氏名: Atsushi  Sakurai
  役職: Corporate VP & GM of Legal Division
  年齢: N/A
  生年: N/A
  報酬: N/A

経営指標：（FUJIFILM Holdings Corporation）
従業員数: 72254
インサイダー保有率: 0.02708
機関投資家保

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `MA_Info_Tool` with `{'symbol': '4901.T'}`



Invoking: `MA_Info_Tool` with `{'symbol': '4901.T'}`



Invoking: `MA_Info_Tool` with `{'symbol': '4901.T'}`



Invoking: `MA_Info_Tool` with `{'symbol': '4901.T'}`



Invoking: `MA_Info_Tool` with `{'symbol': '4901.T'}`



Invoking: `MA_Info_Tool` with `{'symbol': '4901.T'}`


評価情報（FUJIFILM Holdings Corporation）:
時価総額：4534564290560
PER（TTM）：55.86576
予想PER：16.484095
PBR：1.3434975
PEG比率：2.53
企業価値：5026303967232
EV/EBITDA：11.412
EV/売上高：1.648
時価総額/売上高：1.4871461
配当利回り：0.0159
総現金：195267002368
総負債：687606005760
流動比率：1.291
速動比率：0.691
フリーキャッシュフロー：-150838132736
52周最高値：3999.0
52周最低値：2691.3333
平均取引量：3681701
取引量（現在）：5766300
Beta値：0.179
最終配当値：30.0
最終配当日：1743120000
grossMargin：0.40392
EBITDAMargin：0.14444
営業Margin：0.08303001
リスク情報：（FUJIFILM Holdings Corporation）
ベータ係数：0.179
52週高値：3999.0
52週安値：2691.3333
52週変動率：0.26680887
負債比率：20.369
流動比率：1.291
当座比率：0.691
インタレストカバレッジ：None
総負債/EBITDA比率：1.5612329532738762
フリーキャッシュフロー/総負債比率：-0.21936709608765126
株価ボラテ

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `MA_Info_Tool` with `{'symbol': '7203.T'}`



Invoking: `MA_Info_Tool` with `{'symbol': '7203.T'}`



Invoking: `MA_Info_Tool` with `{'symbol': '7203.T'}`


評価情報（Toyota Motor Corporation）:
時価総額：35478925475840
PER（TTM）：7.1531377
予想PER：7.4607
PBR：0.9927556
PEG比率：-11.42
企業価値：61809268621312
EV/EBITDA：8.487
EV/売上高：1.332
時価総額/売上高：0.76485664
配当利回り：0.028499998
総現金：13626135019520
総負債：38914928672768
流動比率：1.183
速動比率：0.986
フリーキャッシュフロー：-1973744697344
52周最高値：3891.0
52周最低値：2183.0
平均取引量：32914188
取引量（現在）：40654400
Beta値：0.434
最終配当値：45.0
最終配当日：1711584000
grossMargin：0.19331
EBITDAMargin：0.157
営業Margin：0.110530004
リスク情報：（Toyota Motor Corporation）
ベータ係数：0.434
52週高値：3891.0
52週安値：2183.0
52週変動率：-0.046170533
負債比率：105.806
流動比率：1.183
当座比率：0.986
インタレストカバレッジ：None
総負債/EBITDA比率：5.343529649421724
フリーキャッシュフロー/総負債比率：-0.05071947359690762
株価ボラティリティ（過去10日）：31909280
株価収益率（P/E比率）：7.1531377
株価純資産倍率（P/B比率）：0.9927556
売上高利益率：0.107080005
売上高成長率：0.122
営業キャッシュフロー：3531816042496
総資産利益率（ROA）：0.03978
株主資本利益率（ROE）：0.15012999